# DNN Model Notebook

```md
@authors: miguelrocha and Grupo 03
```

In [1]:
# Notebook Imports
import numpy as np

from models.dnn_model import NeuralNetwork, hyperparameter_optimization
from helpers.dataset import Dataset
from helpers.enums import ModelRunMode
from helpers.layers import DenseLayer
from helpers.activation import ReLUActivation, SigmoidActivation
from helpers.losses import BinaryCrossEntropy
from helpers.optimizer import Optimizer
from helpers.metrics import accuracy, confusion_matrix, balanced_accuracy, precision_recall_f1
from helpers.enums import ModelType
from helpers.model import save_dnn_model

In [2]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.TRAIN.value
# Prefix for saving the model files
model_prefix = "dnn_model_1"

In [3]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/clean_input_datasets/gpt_vs_human_data_set_inputs.csv"     # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/clean_output_datasets/gpt_vs_human_data_set_outputs.csv"  # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                     # Proportion of the dataset to use as test data
    validation_size = 0.2                                                               # Proportion of the dataset reserved as validation data (used during hyperparameter tuning)
    verbose = True                                                                      # Verbosity level enabler
    random_state=42                                                                     # Seed for reproducible dataset splitting
    max_vocab_size=1000                                                                 # Maximum vocabulary size (None implies no limit)
    min_freq=1                                                                          # Minimum frequency required for a word to be included in the vocabulary
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_2/classify_input_datasets/dataset2_inputs.csv"               # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_2/classify_output_datasets/dataset2_outputs_dnn_model.csv"  # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [4]:
if mode == ModelRunMode.TRAIN.value:
    # Load datasets using TF-IDF vectorization
    X_train, y_train, X_test, y_test, vocab, idf = Dataset.prepare_train_test_tfidf(input_csv=input_csv, output_csv=output_csv, test_size=test_size, random_state=random_state, max_vocab_size=max_vocab_size, min_freq=min_freq)

    # Create Dataset objects for training and test data
    train_ds_full = Dataset(X=X_train, Y=y_train)
    test_ds = Dataset(X=X_test, Y=y_test)

    # Display dimensions of the training and test datasets
    print(f"Train set has {train_ds_full.nrows()} rows and {train_ds_full.ncols()} columns")
    print(f"Test set has {test_ds.nrows()} rows and {test_ds.ncols()} columns\n")

    # Split the full training dataset into training and validation subsets (70% training, 30% validation)
    n_train = train_ds_full.X.shape[0]
    indices = np.arange(n_train)
    np.random.shuffle(indices)
    split_idx = int((1 - validation_size) * n_train)
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]
    train_ds = Dataset(X=train_ds_full.X[train_idx], Y=train_ds_full.Y[train_idx])
    val_ds = Dataset(X=train_ds_full.X[val_idx], Y=train_ds_full.Y[val_idx])

    # More varied epochs to handle both simpler and more complex tasks
    epochs_list = [100, 150]

    # A wider range of batch sizes
    batch_size_list = [32, 64]

    # Learning rate from very small to moderate
    learning_rate_list = [0.001, 0.01]

    # Momentum slightly varied around typical defaults
    momentum_list = [0.8, 0.9]

    # Different hidden layer topologies
    hidden_layers_list = [
        [64],
        [64, 32],
        [128, 64]
    ]

    # Add mild to moderate dropout levels
    dropout_list = [0.0, 0.1]

    # Perform hyperparameter tuning using the training and validation sets
    print("Starting hyperparameter tuning...")
    best_params = hyperparameter_optimization(train_ds, val_ds, epochs_list, batch_size_list, learning_rate_list, momentum_list, hidden_layers_list, dropout_list, n_iter=96)
    print("\nBest hyperparameters:", best_params)

    # Retrain DNN model on the full training data using the best hyperparameters
    final_model = NeuralNetwork(epochs=best_params['epochs'], batch_size=best_params['batch_size'], optimizer=Optimizer(learning_rate=best_params['learning_rate'], momentum=best_params['momentum']), verbose=verbose, loss=BinaryCrossEntropy,  metric=accuracy)

    # Build model Layers
    n_features = train_ds.X.shape[1]
    for i, units in enumerate(best_params['n_hidden']):
        if i == 0:
            final_model.add(DenseLayer(units, (n_features,),dropout_rate=best_params['dropout_rate']))
        else:
            final_model.add(DenseLayer(units,dropout_rate=best_params['dropout_rate']))
        final_model.add(ReLUActivation())
        
    final_model.add(DenseLayer(1))
    final_model.add(SigmoidActivation())

    # Fit model
    final_model.fit(train_ds)

    # Get predictions on the test set
    preds = final_model.predict(test_ds)

    # Convert probabilities to binary class labels using a threshold of 0.5
    preds_binary = (preds > 0.5).astype(int)

    # Evaluate the final model on the test set
    test_acc = final_model.score(test_ds, preds)
    print(f"\nTest accuracy with best hyperparameters: {test_acc:.4f}")

    # Evaluate the final model using binary predictions for the confusion matrix and other metrics
    TP, FP, TN, FN = confusion_matrix(y_test, preds_binary)
    prec, rec, f1 = precision_recall_f1(y_test, preds_binary)
    bal_acc = balanced_accuracy(y_test, preds_binary)

    print("Confusion Matrix: TP={}, FP={}, TN={}, FN={}".format(TP, FP, TN, FN))
    print("Precision = {:.4f}, Recall = {:.4f}, F1 = {:.4f}".format(prec, rec, f1))
    print("Balanced Accuracy = {:.4f}".format(bal_acc))

    # Save the model, plus vocab & idf, so classification can replicate
    save_dnn_model(dnn=final_model, vocab=vocab, idf=idf, model_prefix=model_prefix)
    print(f"DNN Model saved with prefix {model_prefix}")

if mode == ModelRunMode.CLASSIFY.value:
    # Classify new texts using the saved model
    Dataset.classify_texts(input_csv=input_csv, output_csv=output_csv, neural_net_class=NeuralNetwork, model_type=ModelType.DNN ,model_prefix=model_prefix)

Train set has 2838 rows and 1000 columns
Test set has 1215 rows and 1000 columns

Starting hyperparameter tuning...


Hyperparameter Search:   1%|█                                                                                                | 1/96 [00:09<14:20,  9.06s/it]

Parameters: {'epochs': 100, 'batch_size': 32, 'learning_rate': 0.001, 'momentum': 0.9, 'n_hidden': [64], 'dropout_rate': 0.0}  |  Accuracy: 0.9472


Hyperparameter Search:   2%|██                                                                                               | 2/96 [00:22<18:25, 11.76s/it]

Parameters: {'epochs': 150, 'batch_size': 32, 'learning_rate': 0.001, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.0}  |  Accuracy: 0.9630


...

Hyperparameter Search:  99%|███████████████████████████████████████████████████████████████████████████████████████████████ | 95/96 [19:39<00:12, 12.33s/it]

Parameters: {'epochs': 100, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.1}  |  Accuracy: 0.9718


Hyperparameter Search: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [19:46<00:00, 12.36s/it]

Parameters: {'epochs': 100, 'batch_size': 64, 'learning_rate': 0.001, 'momentum': 0.8, 'n_hidden': [64, 32], 'dropout_rate': 0.0}  |  Accuracy: 0.9472

Best Hyperparameters Found: {'epochs': 150, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.9, 'n_hidden': [64], 'dropout_rate': 0.1}
Best Accuracy: 0.9736

Best hyperparameters: {'epochs': 150, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.9, 'n_hidden': [64], 'dropout_rate': 0.1}
Epoch 1/150 - loss: 0.6877 - accuracy: 0.5402
Epoch 2/150 - loss: 0.6547 - accuracy: 0.6299


Epoch 3/150 - loss: 0.6027 - accuracy: 0.7067
Epoch 4/150 - loss: 0.5401 - accuracy: 0.7812
Epoch 5/150 - loss: 0.4701 - accuracy: 0.8219
Epoch 6/150 - loss: 0.3927 - accuracy: 0.8621
Epoch 7/150 - loss: 0.3242 - accuracy: 0.8955
Epoch 8/150 - loss: 0.2706 - accuracy: 0.9187
Epoch 9/150 - loss: 0.2229 - accuracy: 0.9330
Epoch 10/150 - loss: 0.1858 - accuracy: 0.9482
Epoch 11/150 - loss: 0.1642 - accuracy: 0.9464
Epoch 12/150 - loss: 0.1526 - accuracy: 0.9549
Epoch 13/150 - loss: 0.1321 - accuracy: 0.9612
Epoch 14/150 - loss: 0.1185 - accuracy: 0.9701
Epoch 15/150 - loss: 0.0991 - accuracy: 0.9768
Epoch 16/150 - loss: 0.0959 - accuracy: 0.9714
Epoch 17/150 - loss: 0.0818 - accuracy: 0.9826
Epoch 18/150 - loss: 0.0743 - accuracy: 0.9826
Epoch 19/150 - loss: 0.0672 - accuracy: 0.9848
Epoch 20/150 - loss: 0.0654 - accuracy: 0.9830
Epoch 21/150 - loss: 0.0595 - accuracy: 0.9875
Epoch 22/150 - loss: 0.0561 - accuracy: 0.9875
Epoch 23/150 - loss: 0.0564 - accuracy: 0.9857
Epoch 24/150 - loss: